# Barrier clip

### Add column to colonies GeoDataFrame showing if there is the presence of a barrier or not.

Outline of how to implement this:
* Import Delhi/NDMC/JJC dataset **[DONE]**
* Import barrier clip shapefiles **[DONE]**
* Reproject shapefiles to the same CRS **[DONE]**
* Check validity - turn into a function (that can be used for other shapefiles)
    * Check that barrier clip shapefiles are all (poly)lines. 
    * Make sure there are no validity problems.
    * Make sure there are no duplicates.
    * Make sure that no row has None in geometry field.
* Merge barrier clip shapefiles into one shapefile
* Use Spatial Join to create identifier column `barrier`.
    * Create function to do this
    * `barrier` column should be boolean (True or False)
* Export colonies shapefile.
* Open in QGIS and manually inspect features.

In [23]:
import os
from importlib import reload
import pandas as pd
import geopandas as gpd
import spatial_index_utils

In [25]:
reload(spatial_index_utils)

<module 'spatial_index_utils' from 'C:\\Users\\bwbel\\Google Drive\\slum_project\\spatial_index_python\\spatial_index_utils.py'>

### Import Delhi/NDMC/JJC dataset

In [5]:
colonies = gpd.read_file('delhi_ndmc_jjc_corrected.shp')

In [7]:
colonies.head(2)

,AREA,USO_AREA_U,HOUSETAX_C,USO_FINAL,geometry
0,Singhola,3058,H,RV,"POLYGON Z ((1013763.588 1023721.838 0.000, 101..."
1,Indra Colony (Narela),1760,G,RUAC,"POLYGON Z ((1007997.730 1025421.961 0.000, 100..."


### Import barrier shapefiles

In [13]:
barrier_clip_dir = 'Barrier_Clip'

canal_filepath = os.path.join(barrier_clip_dir, 'Canal', 'Canal.shp')
drain_filepath = os.path.join(barrier_clip_dir, 'Drain', 'Major_Drain.shp')
railway_filepath = os.path.join(barrier_clip_dir, 'Railway', 'Railway_Line.shp')

# Check filepath validity before importing shapefiles
print('canal filepath exists:', os.path.exists(canal_filepath))
print('drain filepath exists:', os.path.exists(drain_filepath))
print('railway filepath exists:', os.path.exists(railway_filepath))

canal filepath exists: True
drain filepath exists: True
railway filepath exists: True


In [14]:
canal = gpd.read_file(canal_filepath)
drain = gpd.read_file(drain_filepath)
railway = gpd.read_file(railway_filepath)

In [15]:
canal.head(2)

,FID,CAN_NM,CAN_CLSF,EL_GND,DIST_NM,geometry
0,0,WESTERN YAMUNA CANAL,None,216.682,None,"LINESTRING (77.15632 28.70021, 77.15519 28.70205)"
1,1,WESTERN YAMUNA CANAL,None,221.601,None,"LINESTRING (77.15714 28.69886, 77.15632 28.70021)"


In [16]:
drain.head(2)

,FID,Drain_type,Drain_Name,MAINTAINED,AC_NAME,DISTRICT,geometry
0,0,None,Asola Drain,None,CHHATARPUR,SOUTH,"LINESTRING (77.19468 28.44483, 77.19467 28.445..."
1,1,None,Asola Drain,None,CHHATARPUR,SOUTH,"MULTILINESTRING ((77.18090 28.45571, 77.18086 ..."


In [17]:
railway.head(2)

,FID,RL_ZONE,geometry
0,0,NORTHERN RAILWAY,"LINESTRING (77.25315 28.63192, 77.25308 28.631..."
1,1,NORTHERN RAILWAY,"LINESTRING (77.25309 28.63194, 77.25302 28.63181)"


### Reproject shapefiles to EPSG:3857

In [18]:
colonies = spatial_index_utils.reproject_gdf?

Signature: spatial_index_utils.reproject_gdf(gdf, epsg_code)
Docstring:
Reprojects GeoDataFrame to CRS with EPSG code

Assigns WKT format of projection to EPSG code to GeoDataFrame.

Args:
gdf: GeoDataFrame with any geometry (e.g., Point, Line, Polygon)
epsg_code: EPSG code (integer)

Returns:
    GeoDataFrame reprojected to new crs (based on EPSG code).
File:      c:\users\bwbel\google drive\slum_project\spatial_index_python\spatial_index_utils.py
Type:      function


In [19]:
colonies = spatial_index_utils.reproject_gdf(gdf=colonies, epsg_code=3857)

GeoDataFrame now has the following CRS:

PROJCRS["WGS 84 / Pseudo-Mercator",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["Popular Visualisation Pseudo-Mercator",METHOD["Popular Visualisation Pseudo Mercator",ID["EPSG",1024]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - 85°S to 85°N"],BBOX[-85.06,-180,85.06,180]],ID["EPSG",3857]]


In [20]:
canal = spatial_index_utils.reproject_gdf(gdf=canal, epsg_code=3857)

GeoDataFrame now has the following CRS:

PROJCRS["WGS 84 / Pseudo-Mercator",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["Popular Visualisation Pseudo-Mercator",METHOD["Popular Visualisation Pseudo Mercator",ID["EPSG",1024]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - 85°S to 85°N"],BBOX[-85.06,-180,85.06,180]],ID["EPSG",3857]]


In [21]:
drain = spatial_index_utils.reproject_gdf(gdf=drain, epsg_code=3857)

GeoDataFrame now has the following CRS:

PROJCRS["WGS 84 / Pseudo-Mercator",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["Popular Visualisation Pseudo-Mercator",METHOD["Popular Visualisation Pseudo Mercator",ID["EPSG",1024]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - 85°S to 85°N"],BBOX[-85.06,-180,85.06,180]],ID["EPSG",3857]]


In [22]:
railway = spatial_index_utils.reproject_gdf(gdf=railway, epsg_code=3857)

GeoDataFrame now has the following CRS:

PROJCRS["WGS 84 / Pseudo-Mercator",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["Popular Visualisation Pseudo-Mercator",METHOD["Popular Visualisation Pseudo Mercator",ID["EPSG",1024]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["easting (X)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["northing (Y)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["unknown"],AREA["World - 85°S to 85°N"],BBOX[-85.06,-180,85.06,180]],ID["EPSG",3857]]


### Check validity of shapefiles (and correct)

#### First, check for duplicate rows

In [30]:
gdf_list = {'colonies': colonies, 'canal': canal, 'drain': drain, 'railway': railway}

In [33]:
for gdf in gdf_list:
    print(gdf, 'has duplicate rows:', spatial_index_utils.gdf_has_duplicate_rows(gdf_list[gdf]))

colonies has duplicate rows: False
canal has duplicate rows: False
drain has duplicate rows: False
railway has duplicate rows: False


#### Second, 